In [5]:
import pandas as pd
import numpy as np
import math

# start sklearn
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelBinarizer
# end sklearn


import warnings
warnings.filterwarnings('ignore')

df_train = pd.read_csv('train.csv')

y_train = df_train['averageRating']
df_test = pd.read_csv('test.csv')
ans = df_test['Id']

genres_train = df_train['genres']
genres_test = df_test['genres']

In [6]:
ans = pd.DataFrame(ans)

In [7]:
df_train.drop(columns = ['genres','genderVoters','averageRating','mainWriter','mainDirector','firstShownCountry','prefVotersCountry','mainWriterBirthYear','mainDirectorBirthYear','primaryTitle','originalTitle','isAdult','endYear', 'mainDirectorDeathYear', 'mainWriterDeathYear', 'MainWriterPrimaryProfession','MainDirectorPrimaryProfession','titleLength'], inplace = True)
df_test.drop(columns = ['genres','genderVoters','Id','mainWriter','mainDirector','firstShownCountry','prefVotersCountry','mainWriterBirthYear','mainDirectorBirthYear','primaryTitle','originalTitle','isAdult','endYear', 'mainDirectorDeathYear','mainWriterDeathYear','MainWriterPrimaryProfession','MainDirectorPrimaryProfession','titleLength'], inplace = True)

In [8]:
def special_means_for_keys(df, key_column, res_colum):
    sums = dict.fromkeys(set(df[key_column].values),0)
    num_of_non_nan = dict.fromkeys(set(df[key_column].values),0)

    for i, value in enumerate(df[res_colum]):
        sums[df.iloc[i,0]] += 0 if math.isnan(value) else value
        num_of_non_nan[df.iloc[i,0]] += 0 if math.isnan(value) else 1  
    for key in sums:
        sums[key] = sums[key]/num_of_non_nan[key] if (not num_of_non_nan[key] == 0) else 0  
    return sums

def fillna_for_keys(df,key_column,res_column, dict_values):
    for i in range(len(df[res_column])):
        if math.isnan(df[res_column][i]):
            df[res_column][i] = dict_values[df[key_column][i]]
            
# means = special_means_for_keys(df_train, 'titleType', 'runtimeMinutes')
# fillna_for_keys(df_train,'titleType','runtimeMinutes',means)

# Category_preprocessing

In [9]:
lb = LabelBinarizer()

cat_train = df_train.select_dtypes(include=['object']).copy()
cat_test = df_test.select_dtypes(include=['object']).copy()
cat_test['titleType'] = cat_test['titleType'].astype('category')
cat_train['titleType'] = cat_train['titleType'].astype('category')

# TRAIN linear Binarization
lb_result = lb.fit_transform(cat_train['titleType'])
lb_result_df = pd.DataFrame(lb_result,columns=lb.classes_)
lb_result_df = lb_result_df.join(df_train['worldPromotion'])
# END

# TEST linear Binarization
lb_result = lb.fit_transform(cat_test['titleType'])
lb_result_df_test = pd.DataFrame(lb_result,columns=lb.classes_)
lb_result_df_test = lb_result_df_test.join(df_test['worldPromotion'])
# END

df_train.drop(columns = ['titleType','worldPromotion'], inplace=True)
df_test.drop(columns = ['titleType','worldPromotion'], inplace=True)

df_train['Downloads'].replace({-1:np.nan},inplace=True)
df_train["Downloads"].fillna(df_train["Downloads"].median(), inplace=True)
df_train['runtimeMinutes'].fillna(df_train["runtimeMinutes"].median(),inplace=True)

df_test['Downloads'].replace({-1:np.nan},inplace=True)
df_test["Downloads"].fillna(df_test["Downloads"].median(), inplace=True)
df_test["startYear"].fillna((int)(df_test["startYear"].median()), inplace=True)
df_test['runtimeMinutes'].fillna(df_test["runtimeMinutes"].median(),inplace=True)

In [10]:
data1={'History':0, 'War':0,'Sport':0,'Musical':0,'News':0,'Game-Show':0,'Talk-Show':0,'Adult':0,'Western':0, 'Biography':0,'Reality-TV':0, 'Sci-Fi':0,'Adventure':0,'Family':0,'Animation':0,'Crime':0,'Comedy':0, 'Drama' : 0, 'Action': 0, 'Fantasy': 0, 'Horror': 0, 'Thriller': 0, 'Documentary': 0, 'Mystery': 0, 'Romance': 0,'2Nones':0}

In [11]:
genres = genres_train
ddd = []
nan_handler = 0
for list_genres in genres:
    list_genress = list_genres.replace("[","").replace("]","").replace("'",'').replace(',','')
    list_genress = list_genress.split()
    data = dict.fromkeys(data1,0)
    nones2 = 0
    for i in list_genress:
        if i in data:
            data[i] = 1
        else:
            if i == 'None':
                data['2Nones'] = 1
    nan_handler += 1
    ddd.append(list(data.values()))
    
cat_genres_train = pd.DataFrame(ddd,columns=data.keys())

In [12]:
genres = genres_test
ddd = []
for list_genres in genres:
    list_genress = list_genres.replace("[","").replace("]","").replace("'",'').replace(',','')
    list_genress = list_genress.split()
    data = dict.fromkeys(data1,0)
    nones2 = 0
    for i in list_genress:
        if i in data:
            data[i] = 1
        else:
            if i == 'None':
                data['2Nones'] = 1
    ddd.append(list(data.values()))
    
cat_genres_test = pd.DataFrame(ddd,columns=data.keys())

In [14]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
  display(df_train)
#df_train.info()

# Standardization

In [15]:
sd = StandardScaler()
df_train = pd.DataFrame(sd.fit_transform(df_train))
df_test = pd.DataFrame(sd.transform(df_test))
df_train = df_train.join(lb_result_df)
df_train = df_train.join(cat_genres_train)
df_test = df_test.join(lb_result_df_test)
df_test = df_test.join(cat_genres_test)

# Linear Reg. Learn

In [16]:
lr = linear_model.LinearRegression()
lr.fit(df_train,y_train)
ans = ans.join(pd.DataFrame(lr.predict(df_test)))
ans = ans.rename(index=str, columns={0:'Y'})
ans.to_csv("ans_with_genres_without_Other_with_one_none_col.csv",sep=',', index=False)
